In [2]:
import pandas as pd
import numpy as np
import PyPDF2
import nltk
import re
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
pd.set_option('display.max_rows',None)

# Abrir el archivo del Antiguo Testamento
with open("./Catolico/La biblia antiguo-1-1174.pdf", 'rb') as file_antiguo:
    antiguo_testamento_reader = PyPDF2.PdfReader(file_antiguo)
    paginas_antiguo_testamento = len(antiguo_testamento_reader.pages) 
# Abrir el archivo del Nuevo Testamento
with open("./Catolico/La biblia nuevo-1175-1520.pdf", 'rb') as file_nuevo:
    nuevo_testamento_reader = PyPDF2.PdfReader(file_nuevo)
    paginas_nuevo_testamento = len(nuevo_testamento_reader.pages)  # Use len instead

# Imprimir el número de páginas
print(f"Paginas Antiguo testamento: {paginas_antiguo_testamento}")
print(f"Paginas Nuevo testamento: {paginas_nuevo_testamento}")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Usuario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Paginas Antiguo testamento: 1174
Paginas Nuevo testamento: 346


In [3]:
def extract_text(pdf_path):
    with open(pdf_path, 'rb') as pdf_file:  # Open the file in binary read mode
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        texto_completo = ""
        for pagina in pdf_reader.pages:
            texto_extraido = pagina.extract_text()
            if texto_extraido:
                texto_completo += texto_extraido
        return texto_completo

antiguo_testamento = extract_text("./Catolico/La biblia antiguo-1-1174.pdf")
nuevo_testamento = extract_text("./Catolico/La biblia nuevo-1175-1520.pdf")



In [4]:
#Funciones para Textos
def limpieza_texto(texto):

    # Descargar stopwords en español (si no están descargadas)
    try:
        stopwords.words('spanish')
    except LookupError:
        nltk.download('stopwords')

    # Importar stopwords
    stop_words = stopwords.words('spanish')

    # Tokenización
    tokens = word_tokenize(texto)

    # Eliminar símbolos y convertir a minúsculas
    patron_simbolos = r"[^\w\s]"
    tokens = [re.sub(patron_simbolos, "", token).lower() for token in tokens]

    # Eliminar tildes
    diccionario_tildes = str.maketrans("áéíóú", "aeiou")
    tokens = [token.translate(diccionario_tildes) for token in tokens]

    # Eliminar stopwords y palabras vacías después de eliminar símbolos y tildes
    tokens = [token for token in tokens if token and token not in stop_words]

    # Eliminar números
    patron_numeros = r"\d+"
    tokens = [re.sub(patron_numeros, "", token) for token in tokens]

    # Devolver texto limpio
    texto_limpio = " ".join(tokens)

    return texto_limpio
def frecuencia_textos(texto_tokenizado):
    token = word_tokenize(texto_tokenizado)
    frecuencia_palabras = FreqDist(token)
    df = pd.DataFrame(frecuencia_palabras.items(), columns=['palabra', 'frecuencia'])
    df= df.sort_values(by='frecuencia', ascending=False)
    return df



ANTIGUO TESTAMENTO LIMPIEZA !!!

In [5]:
df_limpio = limpieza_texto(antiguo_testamento)
df = frecuencia_textos(df_limpio)

palabras_a_eliminar = [
    "file",
    "and",
    "cdocuments",
    "electronicosbibliabibliatxt",
    "of",
    "settingsadministradormis",
    "docu",
    "slibros_electronicoslibros",
    "settingsadministrad",
    "tronicoslibros",
    "ctronicoslibros",
    "settingsadministrado"
]

df_antiguo = df[~df['palabra'].isin(palabras_a_eliminar)]
df_antiguo.to_csv('frec_antiguo_test.csv', index=False)


NUEVO TESTAMENTO LIMPIEZA !!!

In [6]:
df_limpio = limpieza_texto(nuevo_testamento)
df = frecuencia_textos(df_limpio)

palabras_a_eliminar = [
    "file",
    "and",
    "cdocuments",
    "electronicosbibliabibliatxt",
    "of",
    "settingsadministradormis",
    "docu",
    "slibros_electronicoslibros",
    "settingsadministrad",
    "tronicoslibros",
    "ctronicoslibros",
    "settingsadministrado"
]

df_nuevo = df[~df['palabra'].isin(palabras_a_eliminar)]
df_nuevo.to_csv('frec_nuevo_test.csv', index=False)


ANALISIS ANTIGUO Y NUEVO TESTAMENTO

In [7]:
df_combinado = pd.merge(df_antiguo, df_nuevo, on='palabra', how='outer', suffixes=('_antiguo_testamento', '_nuevo_testamento'))
df_combinado.fillna(0, inplace=True)
df_combinado['frecuencia_total'] = df_combinado['frecuencia_antiguo_testamento'] + df_combinado['frecuencia_nuevo_testamento']
df_combinado = df_combinado.sort_values(by='frecuencia_total', ascending=False)


ISLAM - CORAN
    

In [11]:
coran = extract_text("./Islam/Coran.pdf")
df_limpio = limpieza_texto(coran)
df_coran = frecuencia_textos(df_limpio)

palabras_a_eliminar = [
    "wwwislamelsal"
]

df_coran = df_coran[~df_coran['palabra'].isin(palabras_a_eliminar)]
df_coran.to_csv('frec_coran.csv', index=False)

ValueError: DataFrame constructor not properly called!

In [10]:
type(coran)

str

HINDUISMO - UPANISHADS

In [ ]:
upanishads = extract_text("./Hinduismo/Upanishads.pdf")
df_limpio = limpieza_texto(upanishads)
df_upanishads = frecuencia_textos(df_limpio)
df_upanishads.to_csv('frec_upanishads.csv', index=False)

JUDAISMO - EL TORÁ

In [ ]:
tora = extract_text("./Judaismo/Torah.pdf")
df_limpio = limpieza_texto(tora)
df_tora = frecuencia_textos(df_limpio)

palabras_a_eliminar = [
    "s","l","n","ki","v","d"
]
df_tora = df_tora[~df_tora['palabra'].isin(palabras_a_eliminar)]
df_tora.to_csv('frec_tora.csv', index=False)